In [75]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
cd "/content/drive/My Drive/Colab Notebooks/Predicta"

[Errno 2] No such file or directory: '/content/drive/My Drive/Colab Notebooks/Predicta'
/content


In [66]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [67]:
df = pd.read_csv("daily_data.csv")
df.head()

,day_id,city_id,temperature_celsius,condition_text,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index,sunrise,sunset
0,D0001,C001,27.0,NaN,6.1,210,1006.0,0.0,54,75,28.0,10.0,6.0,11.9,2,06:04 AM,07:19 PM
1,D0002,C001,22.0,NaN,6.1,170,1006.0,0.0,73,75,24.5,10.0,1.0,23.4,1,06:05 AM,07:18 PM
2,D0003,C001,20.0,Light Rain with Thunder,3.6,10,1011.0,4.5,100,75,20.0,10.0,1.0,12.6,1,06:05 AM,07:18 PM
3,D0004,C001,17.0,Clear and Sunny,6.1,150,1018.0,0.0,88,0,17.0,10.0,1.0,11.2,1,06:06 AM,07:16 PM
4,D0005,C001,18.0,NaN,3.6,92,1019.0,0.0,94,0,18.0,10.0,1.0,9.0,1,06:07 AM,07:15 PM


In [68]:
df.describe()

,temperature_celsius,wind_kph,wind_degree,pressure_mb,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph,air_quality_us-epa-index
count,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000,2893.000000
mean,22.024577,10.304805,158.994469,1013.745938,0.165299,74.773591,32.073280,23.725752,9.745524,2.216730,16.930522,1.599032
std,6.615540,7.264472,103.021634,6.172045,0.937061,19.335724,32.584184,8.877424,2.489918,2.238163,10.541449,1.062513
min,-2.000000,3.600000,1.000000,982.000000,0.000000,4.000000,0.000000,-4.300000,0.000000,1.000000,0.700000,1.000000
25%,17.000000,5.000000,70.000000,1010.000000,0.000000,66.000000,0.000000,17.000000,10.000000,1.000000,9.000000,1.000000
50%,22.000000,8.300000,150.000000,1014.000000,0.000000,78.000000,25.000000,24.500000,10.000000,1.000000,15.100000,1.000000
75%,27.000000,13.000000,236.000000,1018.000000,0.000000,89.000000,68.000000,30.000000,10.000000,1.000000,22.300000,2.000000
max,45.000000,74.200000,360.000000,1036.000000,28.700000,100.000000,100.000000,73.600000,32.000000,10.000000,75.600000,6.000000


In [69]:
# Filter records with non-empty 'condition_text'
df_filtered = df.loc[df['condition_text'].isnull()]
df_filtered.count()

day_id                      2414
city_id                     2414
temperature_celsius         2414
condition_text                 0
wind_kph                    2414
wind_degree                 2414
pressure_mb                 2414
precip_mm                   2414
humidity                    2414
cloud                       2414
feels_like_celsius          2414
visibility_km               2414
uv_index                    2414
gust_kph                    2414
air_quality_us-epa-index    2414
sunrise                     2414
sunset                      2414
dtype: int64

In [70]:
# Function to convert time to minutes past midnight
def time_to_minutes(time_series):
    # Convert to datetime
    time_dt = pd.to_datetime(time_series, format='%I:%M %p')
    # Calculate minutes past midnight
    minutes = time_dt.dt.hour * 60 + time_dt.dt.minute
    return minutes.values


df_filtered.loc[:, 'sunrise_num'] = time_to_minutes(
    df_filtered['sunrise']).astype(int)
df_filtered.loc[:, 'sunset_num'] = time_to_minutes(
    df_filtered['sunset']).astype(int)

<ipython-input-70-ffcc770194e2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'sunrise_num'] = time_to_minutes(
<ipython-input-70-ffcc770194e2>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.loc[:, 'sunset_num'] = time_to_minutes(


In [71]:
# from sklearn.preprocessing import StandardScaler

# # Select numeric columns
# numeric_cols = df_filtered.select_dtypes(include=[np.number]).columns.tolist()

# # Initialize StandardScaler
# scaler = StandardScaler()

# # Apply standardization
# df_filtered.loc[:, numeric_cols] = scaler.fit_transform(df_filtered[numeric_cols])

In [72]:
from sklearn.preprocessing import RobustScaler

# Select numeric columns
numeric_cols = df_filtered.select_dtypes(include=[np.number]).columns.tolist()

# Initialize RobustScaler
scaler = RobustScaler()

# Apply scaling
df_filtered.loc[:, numeric_cols] = scaler.fit_transform(df_filtered[numeric_cols])

In [73]:
# scaler = MinMaxScaler()
# numeric_cols = df_filtered.select_dtypes(include=[np.number]).columns.tolist()

# # Use .loc to avoid SettingWithCopyWarning when scaling numeric columns
# df_filtered.loc[:, numeric_cols] = scaler.fit_transform(
#     df_filtered[numeric_cols])

In [74]:
# Drop the specified columns
df_filtered = df_filtered.drop(columns=['condition_text', 'sunrise', 'sunset'])

# Save the resulting DataFrame to a CSV file with a custom delimiter '-d'
df_filtered.to_csv('filtered_input.csv', index=False)